In [2]:
import torch

# 需要求导的，可以手动定义

In [3]:
# requires_grad=True,表示计算过程中保留参数的梯度
# 方法1
x=torch.randn(3,4,requires_grad=True)
x

tensor([[-0.0407, -0.6299,  0.1328, -0.3474],
        [ 1.7371, -1.0629, -2.1029, -0.6903],
        [-0.1543,  1.1164, -0.2421,  1.7917]], requires_grad=True)

In [4]:
# 方法2
x=torch.randn(3,4)
x.requires_grad=True
x

tensor([[ 2.2543,  1.1056, -1.5862,  1.4370],
        [-1.5083,  0.2337, -0.3712, -0.4232],
        [ 0.3995, -0.0545, -1.4110, -1.0170]], requires_grad=True)

In [5]:
b=torch.randn(3,4,requires_grad=True)
t=x+b
y=t.sum()
y

tensor(-2.9886, grad_fn=<SumBackward0>)

In [6]:
y.backward()

In [8]:
b.grad

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [9]:
x.requires_grad,b.requires_grad,t.requires_grad

(True, True, True)

# 举个例子

In [10]:
# y=wx  z=y+b
x=torch.rand(1)
b=torch.rand(1,requires_grad=True)
w=torch.rand(1,requires_grad=True)
y=w*x
z=y+b

In [11]:
x.requires_grad,b.requires_grad,w.requires_grad,y.requires_grad

(False, True, True, True)

In [26]:
# 反向传播
z.backward(retain_graph=True)#如果不清空梯度会累加起来

In [27]:
w.grad

tensor([0.5165])

In [28]:
b.grad

tensor([5.])

# 线性回归demo

In [59]:
import numpy as np
from numpy import float32
# 构造一组输入数据x和其对应的标签y
x_values=[i for i in range(11)]
x_train=np.array(x_values,dtype=float32)
x_train=x_train.reshape(-1,1)
x_train.shape

(11, 1)

In [60]:
y_values=[2*i+1 for i in x_values]
y_train=np.array(y_values,dtype=float32)
y_train=y_train.reshape(-1,1)
y_train.shape

(11, 1)

In [61]:
import torch.nn as nn

# 线性回归模型
- 线性回归模型其实就是一个不加激活函数的全连接层

In [62]:
class LinearRegressionModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(LinearRegressionModel,self).__init__()
        self.linear=nn.Linear(input_dim,output_dim)

    def forward(self,x):
        out=self.linear(x)
        return out

In [63]:
input_dim=1
output_dim=1
model=LinearRegressionModel(input_dim,output_dim)
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

In [66]:
# 指定参数和损失函数
epochs=1000
learning_rate=0.01
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)
criterion=nn.MSELoss()

In [67]:
# 训练模型
for epoch in range(epochs):
    epoch+=1
    # 转化成tensor
    inputs=torch.from_numpy(x_train)
    labels=torch.from_numpy(y_train)

    # 梯度要清零每一次迭代
    optimizer.zero_grad()

    # 前向传播
    outputs=model(inputs)

    # 计算损失
    loss=criterion(outputs,labels)

    # 反向传播
    loss.backward()

    # 更新权重参数
    optimizer.step()
    if epoch%50==0:
        print('epoch {} , loss {}'.format(epoch,loss.item()))

epoch 50 , loss 0.04780242592096329
epoch 100 , loss 0.027264632284641266
epoch 150 , loss 0.015550735406577587
epoch 200 , loss 0.008869527839124203
epoch 250 , loss 0.005058857146650553
epoch 300 , loss 0.002885378198698163
epoch 350 , loss 0.0016457189340144396
epoch 400 , loss 0.0009386493475176394
epoch 450 , loss 0.0005353694432415068
epoch 500 , loss 0.00030535535188391805
epoch 550 , loss 0.00017416592163499445
epoch 600 , loss 9.93342837318778e-05
epoch 650 , loss 5.6657157983863726e-05
epoch 700 , loss 3.231425580452196e-05
epoch 750 , loss 1.8431570424581878e-05
epoch 800 , loss 1.0512324479350355e-05
epoch 850 , loss 5.996609615976922e-06
epoch 900 , loss 3.4189170037279837e-06
epoch 950 , loss 1.949875922946376e-06
epoch 1000 , loss 1.1128660162285087e-06


In [74]:
# 测试模型预测结果
predicted=model(torch.from_numpy(x_train).requires_grad_()).data.numpy()
predicted

array([[ 0.99803805],
       [ 2.9983206 ],
       [ 4.998603  ],
       [ 6.998886  ],
       [ 8.999168  ],
       [10.999451  ],
       [12.999734  ],
       [15.000016  ],
       [17.000298  ],
       [19.00058   ],
       [21.000862  ]], dtype=float32)

In [ ]:
# 模型的保存与读取

In [75]:
torch.save(model.state_dict(),'model.pkl')

In [77]:
model.load_state_dict(torch.load('model.pkl'))

<All keys matched successfully>

In [ ]:
# 使用GPU进行训练

In [78]:
class LinearRegressionModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(LinearRegressionModel,self).__init__()
        self.linear=nn.Linear(input_dim,output_dim)

    def forward(self,x):
        out=self.linear(x)
        return out

input_dim=1
output_dim=1
model=LinearRegressionModel(input_dim,output_dim)

# 使用gpu
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs=1000
learning_rate=0.01
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)
criterion=nn.MSELoss()

# 训练模型
for epoch in range(epochs):
    epoch+=1
    # 转化成tensor,使用gpu
    inputs=torch.from_numpy(x_train).to(device)
    labels=torch.from_numpy(y_train).to(device)

    # 梯度要清零每一次迭代
    optimizer.zero_grad()

    # 前向传播
    outputs=model(inputs)

    # 计算损失
    loss=criterion(outputs,labels)

    # 反向传播
    loss.backward()

    # 更新权重参数
    optimizer.step()
    if epoch%50==0:
        print('epoch {} , loss {}'.format(epoch,loss.item()))

epoch 50 , loss 0.1486671268939972
epoch 100 , loss 0.08479418605566025
epoch 150 , loss 0.048363447189331055
epoch 200 , loss 0.027584658935666084
epoch 250 , loss 0.015733283013105392
epoch 300 , loss 0.008973699063062668
epoch 350 , loss 0.005118268076330423
epoch 400 , loss 0.0029192534275352955
epoch 450 , loss 0.0016650266479700804
epoch 500 , loss 0.0009496742277406156
epoch 550 , loss 0.0005416572676040232
epoch 600 , loss 0.00030893986695446074
epoch 650 , loss 0.00017621157167013735
epoch 700 , loss 0.00010050376295112073
epoch 750 , loss 5.732156569138169e-05
epoch 800 , loss 3.269395892857574e-05
epoch 850 , loss 1.86480119737098e-05
epoch 900 , loss 1.063610125129344e-05
epoch 950 , loss 6.066492460377049e-06
epoch 1000 , loss 3.460440211711102e-06
